In [13]:
import os
import subprocess
import requests
import time

# Step 1: Ensure Gunicorn is installed
try:
    import gunicorn
    print("Step 1: Gunicorn is installed.")
except ImportError as e:
    print("Step 1 failed: Gunicorn is not installed. Please install it using pip.")
    raise e

# Step 2: Define a simple Flask application
app_code = """
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello, World!"

if __name__ == "__main__":
    app.run()
"""
try:
    with open("myapp.py", "w") as f:
        f.write(app_code)
    print("Step 2: Flask application created successfully.")
except Exception as e:
    print(f"Step 2 failed: {str(e)}")
    raise e

# Step 3: Function to stop Gunicorn
def stop_gunicorn(process):
    try:
        process.terminate()
        process.wait()
        print("Step 6: Gunicorn stopped successfully.")
    except Exception as e:
        print(f"Step 6 failed: {str(e)}")
        raise e

# Step 4: Function to start Gunicorn on a different port if 8000 is in use
def start_gunicorn_with_logs():
    port = 8003
    while True:
        try:
            command = f"gunicorn --bind 127.0.0.1:{port} --access-logfile access.log --error-logfile error.log --log-level debug myapp:app"
            with open("gunicorn.log", "w") as f:
                process = subprocess.Popen(command, stdout=f, stderr=f, shell=True)
            print(f"Step 4: Gunicorn started successfully on port {port} with debug logging.")
            return process, port
        except subprocess.CalledProcessError as e:
            if "Address already in use" in str(e):
                print(f"Port {port} is already in use, trying next port...")
                port += 1
            else:
                print(f"Step 4 failed: {str(e)}")
                raise e

# Step 5: Ensure no previous log files exist
try:
    log_files = ["access.log", "error.log", "gunicorn.log"]
    for log_file in log_files:
        if os.path.exists(log_file):
            os.remove(log_file)
    print("Step 5: Previous log files removed.")
except Exception as e:
    print(f"Step 5 failed: {str(e)}")
    raise e

# Step 6: Start Gunicorn and generate logs
try:
    process, port = start_gunicorn_with_logs()
    time.sleep(3)  # Give Gunicorn time to start

    # Send a request to generate log entries
    response = requests.get(f"http://127.0.0.1:{port}")
    assert response.status_code == 200, "Failed to connect to the server."
    print("Step 6: Request sent to server successfully.")
except Exception as e:
    print(f"Step 6 failed: {str(e)}")
    stop_gunicorn(process)
    raise e

# Step 7: Check if log files were created and contain content
try:
    if os.path.exists("access.log") and os.path.getsize("access.log") > 0:
        print("Step 7: Access log exists and has content.")
    else:
        print("Step 7 failed: Access log does not exist or is empty.")
    
    if os.path.exists("error.log"):
        print("Step 7: Error log exists.")
    else:
        print("Step 7 failed: Error log does not exist.")
except Exception as e:
    print(f"Step 7 failed: {str(e)}")
    raise e

# Step 8: Stop Gunicorn and clean up
try:
    stop_gunicorn(process)
    
    # Inspect Gunicorn log file after process has been stopped
    with open("gunicorn.log", "r") as f:
        print("Gunicorn log output:\n", f.read())
    
    # Cleanup
    for log_file in log_files:
        if os.path.exists(log_file):
            os.remove(log_file)
    if os.path.exists("myapp.py"):
        os.remove("myapp.py")
    print("Step 8: Cleanup completed successfully.")
except Exception as e:
    print(f"Step 8 failed: {str(e)}")
    raise e

# Final Confirmation
print("All tests for Gunicorn completed successfully.")


Step 1: Gunicorn is installed.
Step 2: Flask application created successfully.
Step 5: Previous log files removed.
Step 4: Gunicorn started successfully on port 8003 with debug logging.
Step 6: Request sent to server successfully.
Step 7: Access log exists and has content.
Step 7: Error log exists.
Step 6: Gunicorn stopped successfully.
Gunicorn log output:
 
Step 8: Cleanup completed successfully.
All tests for Gunicorn completed successfully.
